In [1]:
import random
import json
import pickle
import numpy as np

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer

In [3]:
from tensorflow.keras.models import load_model

In [4]:
lemmatizer=WordNetLemmatizer()

In [8]:
intents=json.loads(open('intents.json').read())
words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))

In [10]:
model=load_model('Chatbot_model.h5')

In [11]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [12]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
    return(np.array(bag))


In [14]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [15]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [16]:
print("Your Assistant is active!!")

Your Assistant is active!!


In [ ]:
while True:
    message=input("")
    ints=predict_class(message,model)
    res=getResponse(ints,intents)
    print(res)

Hey
1/1 [==============================] - 0s 173ms/step
Hello! How can I assist you?
what is yoo name
1/1 [==============================] - 0s 26ms/step
I'm just a virtual assistant and don't have a personal name. How can I assist you?
tell me some sports recom
1/1 [==============================] - 0s 26ms/step
Sure! How about trying out jogging, swimming, or cycling? They are great for overall fitness.
good morning
1/1 [==============================] - 0s 27ms/step
Hey! How can I help you?
see ya
1/1 [==============================] - 0s 29ms/step
Goodbye! Have a great day!
